## TASK 4: PREPARE THE PROPOSALS FOR TRAINING

Summary of procedure:

- Iterate through all images and their annotations.

- Calculate the IoU for each proposal and assign the corresponding label.

- Store the information about proposals and labels in a format useful for training.

- Save the prepared data for use in the object detection model.

In [ ]:
import os
import cv2
import selectivesearch
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import numpy as np
#import torch 
# from torch.utils.data import Dataset

In [ ]:
# Function to prepare proposals for PyTorch
def prepare_proposals(images_path, annotations_path, proposals_per_image, iou_threshold=0.5):
    proposal_data = []

    for filename in os.listdir(images_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(images_path, filename)
            annotation_file = os.path.join(annotations_path, filename.replace('.jpg', '.xml'))
            
            image = cv2.imread(image_path)
            ground_truth_boxes = parse_xml(annotation_file)
            proposals = get_proposals(image, proposals_per_image)
            
            for (x, y, w, h) in proposals:
                proposal_box = (x, y, x + w, y + h)
                max_iou = 0  # Initialize maximum IoU for the current proposal
                
                for gt_box in ground_truth_boxes:
                    iou = calculate_iou(proposal_box, gt_box)
                    if iou > max_iou:
                        max_iou = iou  # Update maximum IoU if a new higher value is found
                
                if max_iou >= iou_threshold:
                    label = 1  # Pothole
                else:
                    label = 0  # Background

                proposal_data.append({
                    'image_filename': filename,
                    'proposal': (x, y, w, h),
                    'label': label
                })

    return proposal_data

# Execute the function
# prepare_proposals(images_path, annotations_path, 100)

We use the above function in a class for generating the Pothole images.

In [ ]:
# Pothole Dataset Class
class PotholeDataset(Dataset):
    def __init__(self, images_path, annotations_path, proposals_per_image, transform=None, iou_threshold=0.5):
        self.images_path = images_path
        self.annotations_path = annotations_path
        self.proposals_per_image = proposals_per_image
        self.transform = transform
        self.iou_threshold = iou_threshold
        self.proposal_data = self.prepare_proposals()

    def prepare_proposals(self):
        proposal_data = []
        
        for filename in os.listdir(self.images_path):
            if filename.endswith(".jpg"):
                image_path = os.path.join(self.images_path, filename)
                annotation_file = os.path.join(self.annotations_path, filename.replace('.jpg', '.xml'))
                
                image = cv2.imread(image_path)
                ground_truth_boxes = parse_xml(annotation_file)
                proposals = self.get_proposals(image)
                
                for (x, y, w, h) in proposals:
                    proposal_box = (x, y, x + w, y + h)
                    max_iou = 0  # Initialize maximum IoU for the current proposal
                    
                    for gt_box in ground_truth_boxes:
                        iou = calculate_iou(proposal_box, gt_box)
                        if iou > max_iou:
                            max_iou = iou  # Update maximum IoU if a new higher value is found
                    
                    # Assign label based on maximum IoU found
                    label = 1 if max_iou >= self.iou_threshold else 0

                    proposal_data.append({
                        'image_filename': filename,
                        'proposal': (x, y, w, h),
                        'label': label
                    })

        return proposal_data

    def get_proposals(self, image):
        # Placeholder function for proposal generation (to be implemented)
        proposals = []  # Replace with actual proposal generation code
        return proposals[:self.proposals_per_image]

    def __len__(self):
        return len(self.proposal_data)

    def __getitem__(self, idx):
        data = self.proposal_data[idx]
        image_filename = data['image_filename']
        proposal = data['proposal']
        label = data['label']

        # Load image
        image_path = os.path.join(self.images_path, image_filename)
        image = cv2.imread(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image, proposal, label